In [ ]:
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from skimage import util
from math import exp, log
from tensorflow.python.keras.engine.base_layer import Layer
from tensorflow.keras import layers
from matplotlib import rc
rc('mathtext', default='regular')

In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'imagenette',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imagenette/full-size-v2/0.1.0.incompleteBM3XZM/imagenette-train.tfrecord


  0%|          | 0/9469 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imagenette/full-size-v2/0.1.0.incompleteBM3XZM/imagenette-validation.tfrecord


  0%|          | 0/3925 [00:00<?, ? examples/s]

Dataset imagenette downloaded and prepared to /root/tensorflow_datasets/imagenette/full-size-v2/0.1.0. Subsequent calls will reuse this data.


In [ ]:
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

In [ ]:
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, shuffle=False, augment=False, batch_flag=True):
    # Resize and rescale all datasets.
    ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1000)

    # Batch all datasets.
    if batch_flag:
        ds = ds.batch(batch_size)
    else:
        ds = ds.batch(947)

    # Use data augmentation only on the training set.
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                    num_parallel_calls=AUTOTUNE)

    # Use buffered prefetching on all datasets.
    return ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
train_ds = prepare(train_ds, shuffle=True, augment=False)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds, batch_flag =False)

In [ ]:
kernelsize = 3
filters_num1 = 16
filters_num2 = 32
filters_num3 = 32
trust_res = []
train_acc = []
train_loss = []
y_update_wb = []
y_N_opinion = []
y_pred_list = []
y_true_op = [1.0, 0.0, 0.0, 0.5]
x_data_op = [1.0, 0.0, 0.0, 0.5]
w_initial = [0.0, 0.0, 1.0, 0.5]
b_initial = [0.0, 0.0, 1.0, 0.5]

In [ ]:
# functional API
inputs = tf.keras.Input(shape=(180, 180, 3))
x = tf.keras.layers.Conv2D(
            filters=filters_num1,             # 卷积层神经元（卷积核）数目
            kernel_size=[kernelsize, kernelsize],     # 感受野大小
            padding='same',         # padding策略（vaild 或 same）
            activation=tf.nn.relu   # 激活函数
        )(inputs)
x = tf.keras.layers.Conv2D(
            filters=filters_num2,
            kernel_size=[kernelsize, kernelsize],
            padding='same',
            activation=tf.nn.relu
        )(x)
x = tf.keras.layers.Conv2D(
            filters=filters_num3,
            kernel_size=[kernelsize, kernelsize],
            padding='same',
            activation=tf.nn.relu
        )(x)
x = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(units=64, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(units=10)(x)
outputs = tf.keras.layers.Softmax()(x)   

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 180, 180, 32)      4640      
                                                                 
 conv2d_2 (Conv2D)           (None, 180, 180, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 259200)            0         
                                                             

In [ ]:
# forward opinion prop
def mul_scale(W_x):
    mul_u = 1.0
    for i in range(len(W_x)):
        mul_u = mul_u * W_x[i]
        if mul_u <= 1e-24: 
            mul_u = mul_u*1e25
    return mul_u

def fusion(W_x): # W_x array

    deno = 0.0
    mole = 0.0
    full_multi = mul_scale(W_x[:,2])
    for i in range(len(W_x[:,2])):
        deno = deno + full_multi/W_x[:,2][i]
        mole = mole + (full_multi/W_x[:,2][i])*W_x[:,0][i]
    W_b = mole/deno  #  change
    W_u = (len(W_x)*full_multi)/deno   # change
    W_a = sum(W_x[:,3])/len(W_x)  

#     W_b = sum(1/len(W_x)*W_x[:,0])
#     W_u = 0.0
#     W_a = sum(1/len(W_x)*W_x[:,3])
        
    return [W_b,1-W_b-W_u,W_u,W_a]

def avg_fusion_dense(W_wx):
    """
    input - (node_num, 4)
    
    return opinion - 4,32,50
    """
    W_wx = np.moveaxis(W_wx, 0, -1).astype(np.float64) # 4，node_num
    num_para = W_wx.shape[0]
    b_combine, u_combine, a_combine = W_wx[0], W_wx[2], W_wx[3]
    u_combine_recip = (np.zeros(u_combine.shape)+1)/u_combine
    
    numerator = np.sum(b_combine * u_combine_recip) # (32, 50)
    denominator = np.sum(u_combine_recip)
    
    b_fusion = numerator / denominator
    u_fusion = num_para / denominator
    a_fusion = np.sum(a_combine) / num_para
    out_opinion = np.array([b_fusion, 1-b_fusion-u_fusion, u_fusion, a_fusion]).astype(np.float32)
    
    return np.moveaxis(out_opinion, 0, -1)

def fusion_2(W_x, W_y):
    if W_x[2]!=0 or W_y[2]!=0:
        W_b = (W_x[0]*W_y[2]+W_y[0]*W_x[2])/(W_x[2]+W_y[2])
        W_u = 2*W_x[2]*W_y[2]/(W_x[2]+W_y[2])
        W_a = (W_x[3]+W_y[3])/2  
    elif W_x[2]==0 and W_y[2]==0:
        W_b = 0.5*W_x[0]+0.5*W_y[0]
        W_u = 0
        W_a = 0.5*W_x[3]+0.5*W_y[3]
    return [W_b,1-W_b-W_u,W_u,W_a]

def multi(W_x, W_y): # 
    W_b = W_x[0]*W_y[0]+((1-W_x[3])*W_y[3]*W_x[0]*W_y[2]+W_x[3]*(1-W_y[3])*W_y[0]*W_x[2])/(1-W_x[3]*W_y[3])
    W_d = W_x[1]+W_y[1]-W_x[1]*W_y[1]
    W_u = W_x[2]*W_y[2]+((1-W_y[3])*W_x[0]*W_y[2]+(1-W_x[3])*W_y[0]*W_x[2])/(1-W_x[3]*W_y[3])
    W_a = W_x[3]*W_y[3]
    return [W_b,W_d,W_u,W_a]

def get_opinion(r,num_batches): # r -> W
    return [r/(num_batches+2), (num_batches-r)/(num_batches+2), 2/(num_batches+2), 0.5]

def evidence_collect(y, y_pred):
    r = 0
    s = 0
    r_list = [0]*len(y_pred[0])
    s_list = [0]*len(y_pred[0])
    
    for j in range(len(y_pred)):
        for i in range(len(y_pred[0])):
            if i == y[j]:
                if y_pred[j][i] > 0.95:
                    r_list[i]+=1
                    r+=1
                else:
                    s+=1
                    s_list[i]+=1
            else:
                if y_pred[j][i] < 0.05:
                    r_list[i]+=1
                else:
                    s_list[i]+=1
                    
    y_N_op = []
    for i in range(len(r_list)):
        y_N_op.append([r_list[i]/(r_list[i]+s_list[i]+2), 
                       s_list[i]/(r_list[i]+s_list[i]+2), 2/(r_list[i]+s_list[i]+2), 0.5])

    
    return [r/(r+s+2), s/(r+s+2), 2/(r+s+2), 0.5], y_N_op

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [ ]:
#@title
# training process
batch_index = 0
for i in range(5):
    for X,y in train_ds:
        print('# of batch:',batch_index)
        with tf.GradientTape() as tape:
            y_pred = model(X)
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        grads = tape.gradient(loss, model.variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

        sparse_categorical_accuracy.update_state(y_true=y, y_pred=y_pred)
        y_pred_list.append(y_pred)
        train_loss.append(tf.reduce_mean(loss).numpy())
        train_acc.append(sparse_categorical_accuracy.result().numpy())
        print("train accuracy:",sparse_categorical_accuracy.result().numpy())

        y_N_update, y_N_op = evidence_collect(y, y_pred)
        y_N_opinion.append(y_N_op)
        y_update_wb.append(y_N_update)

        batch_index+=1

In [ ]:
result = []

In [ ]:
for X_test,y_test in test_ds:
    y_pred_test = model.predict(X_test)
    sparse_categorical_accuracy.reset_states()
    sparse_categorical_accuracy.update_state(y_true=y_test, y_pred=y_pred_test)
    print("test accuracy:",sparse_categorical_accuracy.result().numpy())

test accuracy: 0.5248152


In [ ]:
# compute Backward opinion of neuron W_N_Y  
W_N_Y=[]
for i in range(len(y_N_opinion)):
    W_N=[]
    for j in y_N_opinion[i]:
        W_N.append(multi(j,y_true_op)) # change when add flaw in label
    W_N_Y.append(W_N)

In [ ]:
# compute forward prop opinion W_NX
W_NX_res=[]
x_data_op_list = []
conv1_out_list = []
conv2_out_list = []
conv3_out_list = []
dense1_out_list = []
dense2_out_list = []
flatten_num = model.layers[-4].output_shape[1]

for i in range(len(y_update_wb)):
    W_w=y_update_wb[i]
    W_b=y_update_wb[i]

    W_xw=[]
    W_xw.append(W_b)
    for j in range(kernelsize*kernelsize*3):
        W_xw.append(multi(W_w,x_data_op))
    conv1_out = fusion(np.array(W_xw))
    conv1_out_list.append(conv1_out)

    W_xw=[]
    W_xw.append(W_b)
    for j in range(kernelsize*kernelsize*filters_num1):
        W_xw.append(multi(W_w,conv1_out))
    conv2_out = fusion(np.array(W_xw))
    conv2_out_list.append(conv2_out)

    W_xw=[]
    W_xw.append(W_b)
    for j in range(kernelsize*kernelsize*filters_num2):
        W_xw.append(multi(W_w,conv2_out))
    conv3_out = fusion(np.array(W_xw))
    conv3_out_list.append(conv3_out)

In [ ]:
# compute last layer output opinion and trust
W_XY = []
for i in range(len(conv3_out_list)):
    W_XY_one = []
    for j in range(5):
        W_XY_one.append(fusion_2(conv3_out_list[i],W_N_Y[i][j]))
    W_XY.append(W_XY_one)
    
W_NN = []
for i in range(len(W_XY)):
    W_NN.append(avg_fusion_dense(np.array(W_XY[i])))

W_NN_trust=[]
for i in W_NN:
    W_NN_trust.append(i[0]+i[2]*i[3])

In [ ]:
fig = plt.figure()
plt.xlabel('Training batch')
plt.ylabel('Trust / accuracy')
plt.plot(W_NN_trust, color='green', label='Trust')
plt.plot(train_acc, color='red', label='Accuracy')
plt.legend()

In [ ]:
plt.plot([i for i in range(len(W_NN))], [i[0] for i in W_NN], color='green', label='belief')
plt.plot([i for i in range(len(W_NN))], [i[1] for i in W_NN], color='red', label='disbelief')
plt.plot([i for i in range(len(W_NN))], [i[2] for i in W_NN],  color='skyblue', label='uncertainty')
plt.plot([i for i in range(len(W_NN))], [i[3] for i in W_NN], color='blue', label='base rate')
plt.legend() # 显示图例

plt.xlabel('Training Episode')
plt.ylabel('Mass')
plt.show()

In [ ]:
import pickle
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
 
def load_variavle(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r

In [ ]:
save_variable(W_NN, 'VGG3_opinion_training')
save_variable(W_NN_trust, 'VGG3_trust_training')
save_variable(train_acc, 'VGG3_acc_training')

'VGG3_acc_training'